In [4]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from sklearn.tree import DecisionTreeClassifier
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import SVC
import pickle


In [16]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]==0):
            
            #if the predicited class is negative
            if(Y_pred[i]==0):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]==4):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Predicted Negative','Predicted Positive'])
    
    Confusion_Matrix.rename(index={0: "Actual Negative", 1: "Actual Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)


In [2]:
data_train = pd.read_csv('sentiment_train.csv')
data_test = pd.read_csv('sentiment_test.csv')

In [8]:
file=open("TrainFeatureFinal","rb")
Train=pickle.load(file)
file1=open("TestFeatureFinal","rb")
Test=pickle.load(file1)
file.close()
file1.close()

In [10]:
Username={}
GivenTweets=data_train['5'][0:50000]
for i in tqdm(range(len(GivenTweets))):
    tweet=GivenTweets[i].split(" ")
    for word in tweet:
        if(re.match("@[a-z_0-9]",word.lower())):
            if(word in Username):
                Username[word]+=1
            else:
                Username[word]=1


100%|██████████| 50000/50000 [00:00<00:00, 54414.54it/s]


In [12]:
for i in Username:
    if(Username[i]>100):
        print(i)

@mileycyrus
@tommcfly
@ddlovato


In [ ]:
username1=np.zeros()

In [13]:
DTmodel=DecisionTreeClassifier()

In [15]:
DTmodel.fit(Train,data_train['0'][0:50000])

DecisionTreeClassifier()

In [17]:
ypred=DTmodel.predict(Test)

In [18]:
Y_Actual=data_test['0']

In [20]:
func_Eval(Y_Actual,ypred)

-----------------
Accuracy: 0.7348666666666667
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.7364461846051317
-----------------
Recall: 0.7342563829963953
-----------------
F1 Score: 0.7353496535489399

For the Negative CLass, Label:0
-----------------
Precision: 0.7332860953651217
-----------------
Recall: 0.7354810120232104
-----------------
F1 Score: 0.7343819136590944

--------
Macro Average F1 0.7348657836040171
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative              175930               63274
Actual Positive               63990              176806


In [21]:
TrainUser1=np.zeros(len(GivenTweets))
TrainUser2=np.zeros(len(GivenTweets))
TrainUser3=np.zeros(len(GivenTweets))
for i in tqdm(range(len(GivenTweets))):
    tweet=GivenTweets[i].split(" ")
    for word in tweet:
        if("@mileycyrus",word.lower()):
            TrainUser1[i]=1
        if("@tommcfly",word.lower()):
            TrainUser2[i]=1
        if("@ddlovato",word.lower()):
            TrainUser3[i]=1

TestTweets=data_test['5']
TestUser1=np.zeros(len(TestTweets))
TestUser2=np.zeros(len(TestTweets))
TestUser3=np.zeros(len(TestTweets))
for i in tqdm(range(len(TestTweets))):
    tweet=TestTweets[i].split(" ")
    for word in tweet:
        if("@mileycyrus",word.lower()):
            TestUser1[i]=1
        if("@tommcfly",word.lower()):
            TestUser2[i]=1
        if("@ddlovato",word.lower()):
            TestUser3[i]=1


100%|██████████| 480000/480000 [00:06<00:00, 72317.74it/s]


In [27]:
trainAdd=pd.DataFrame(TrainUser1)
trainAdd["user2"]=pd.DataFrame(TrainUser2)
trainAdd["user3"]=pd.DataFrame(TrainUser3)

In [28]:
testAdd=pd.DataFrame(TestUser1)
testAdd["user2"]=pd.DataFrame(TestUser2)
testAdd["user3"]=pd.DataFrame(TestUser3)

In [29]:
trainAdd=csr_matrix(trainAdd)
testAdd=csr_matrix(testAdd)
Train1=hstack([Train,trainAdd])
Test1=hstack([Test,testAdd])


In [30]:
DTmodel=DecisionTreeClassifier()
DTmodel.fit(Train1,data_train['0'][0:50000])
ypred1=DTmodel.predict(Test1)

In [31]:
func_Eval(Y_Actual,ypred1)

-----------------
Accuracy: 0.7366791666666667
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.7399471454339528
-----------------
Recall: 0.7325536969052642
-----------------
F1 Score: 0.73623185986235

For the Negative CLass, Label:0
-----------------
Precision: 0.7334547411117089
-----------------
Recall: 0.7408320931088109
-----------------
F1 Score: 0.737124958923825

--------
Macro Average F1 0.7366784093930875
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative              177210               61994
Actual Positive               64400              176396
